# **BERTopic on German & French Amazon Reviews**

In our previous experiment, we proved that **KeyBERT-Inspired representations** significantly outperform standard frequency-based labels (c-TF-IDF) for English news. They produce cleaner, more semantic topic descriptions.

In this notebook, we test if that same strategy holds up in a multilingual environment using **Amazon Reviews** in **German** and **French**. Reviews are \"messy\" data filled with slang, typos, and strong sentiment. We will train separate BERTopic models for each language and apply the KeyBERT-inspired representation to see if we can extract clear, interpretable product categories from raw customer feedback.


In [38]:
!pip install --upgrade pip
!pip install -r requirements.txt
!python -m spacy download de_core_news_lg
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 9.1 MB/s  0:00:580:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 8.8 MB/s  0:01:020:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [52]:
from datasets import load_dataset

ds_german = load_dataset("SetFit/amazon_reviews_multi_de")
ds_french = load_dataset("SetFit/amazon_reviews_multi_fr")
docs_train_german = ds_german["train"]["text"]
categories_train_german = ds_german["train"]["label_text"]

docs_train_french = ds_french["train"]["text"]
categories_train_french = ds_french["train"]["label_text"]

Repo card metadata block was not found. Setting CardData to empty.


### Data Preparation
Based on the results of the previous experiment, we decided to use **lemmatization** this time to improve BERTopic. We did so because of the following reasons:
- we can remove the noise (words like Produkte, Produkten, Produktes will be merged into one -> Produkt)
- topics should be more semantically consistent (because of fewer repeated word variants)
- smaller vocabulary means more stable clustering

In [65]:
import spacy
from nltk.stem.snowball import SnowballStemmer

nlp_de = spacy.load("de_core_news_lg")
nlp_fr = spacy.load("fr_core_news_lg")

stemmer_de = SnowballStemmer("german")
stemmer_fr = SnowballStemmer("french")

def aggressive_clean_and_stem(text_list: list[str], lang: str) -> list[str]:
    """
    1. Cleans technical garbage (<S0>, symbols)
    2. Uses SpaCy to filter stopwords/punctuation
    3. Uses Snowball to aggressively chop words to their root
    """
    if lang == "de":
        nlp = nlp_de
        stemmer = stemmer_de
    elif lang == "fr":
        nlp = nlp_fr
        stemmer = stemmer_fr
    else:
        raise ValueError("Lang must be 'de' or 'fr'")

    cleaned_docs = []

    for doc in nlp.pipe(text_list, batch_size=2000, disable=["parser", "ner"]):

        tokens_to_keep = []

        for token in doc:
            if token.is_alpha and not token.is_stop and len(token.text) > 2:
                stem = stemmer.stem(token.text.lower())
                tokens_to_keep.append(stem)

        cleaned_docs.append(" ".join(tokens_to_keep))

    return cleaned_docs

print("Processing German Docs (this might take a minute)...")
stemmed_german = aggressive_clean_and_stem(docs_train_german[:30000], "de")

print("Processing French Docs...")
stemmed_french = aggressive_clean_and_stem(docs_train_french[:30000], "fr")

Processing German Docs (this might take a minute)...
Processing French Docs...

✅ Done.


### **Set up**
##### **Baseline BERTopic model for German reviews**

Here, we train a BERTopic model on German Amazon reviews. We use the multilingual embedding setting and restrict the topic categorization to 10 topics with a minimum topic size of 20 documents.

In [66]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-mpnet-base-v2")
vectorizer_model = CountVectorizer(stop_words="english")

In [67]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

representation_model_german = KeyBERTInspired()

topic_model_german = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,
    nr_topics=10,
    representation_model=representation_model_german
)

topics_base_german, probs_base_german = topic_model_german.fit_transform(stemmed_german)

2025-12-07 02:54:24,493 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 938/938 [04:25<00:00,  3.53it/s] 
2025-12-07 02:58:50,628 - BERTopic - Embedding - Completed ✓
2025-12-07 02:58:50,634 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 02:59:01,004 - BERTopic - Dimensionality - Completed ✓
2025-12-07 02:59:01,019 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 03:00:03,193 - BERTopic - Cluster - Completed ✓
2025-12-07 03:00:03,217 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-07 03:00:04,117 - BERTopic - Representation - Completed ✓
2025-12-07 03:00:04,122 - BERTopic - Topic reduction - Reducing number of topics
2025-12-07 03:00:04,180 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 03:00:19,156 - BERTopic - Representation - Completed ✓
2025-12-07 03:00:19,260 - BERTopic - Topic reduction - R

##### **Baseline BERTopic model for French reviews**

We repeat the same setup for the French Amazon reviews, again using a multilingual model and the KeyBERTInspired representation.

In [70]:
representation_model_french = KeyBERTInspired()

topic_model_french = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,
    nr_topics=10,
    representation_model=representation_model_french
)

topics_base_french, probs_base_french = topic_model_french.fit_transform(stemmed_french)

2025-12-07 03:01:50,842 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 938/938 [03:07<00:00,  5.00it/s]
2025-12-07 03:04:58,762 - BERTopic - Embedding - Completed ✓
2025-12-07 03:04:58,767 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 03:05:07,981 - BERTopic - Dimensionality - Completed ✓
2025-12-07 03:05:07,989 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 03:05:46,554 - BERTopic - Cluster - Completed ✓
2025-12-07 03:05:46,558 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-07 03:05:47,317 - BERTopic - Representation - Completed ✓
2025-12-07 03:05:47,317 - BERTopic - Topic reduction - Reducing number of topics
2025-12-07 03:05:47,356 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 03:05:53,407 - BERTopic - Representation - Completed ✓
2025-12-07 03:05:53,430 - BERTopic - Topic reduction - Re

### **Inspecting computed topics for German**

In [71]:
info_df = topic_model_german.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(6))

,Topic,Count,Top Keywords
1,0,20505,"bild, gekauft, angekomm, verkauf, geliefert, enttauscht"
2,1,537,"angekomm, artikel, angegeb, angeb, aktivkohl, bestellt"
3,2,167,"kaputtgegang, kaputt, gegang, kupplung, kapputt, gesorgt"
4,3,151,"erhalt, wartet, wartezeit, erzielt, war, erwunscht"
5,4,70,"funktioniert, funktionstucht, funktionert, funktioni, funktionsfah, funtioniert"
6,5,37,"empfehl, emfehl, empfehlenswert, empfehlungswert, kaufempfehl, empfind"


The model successfully identifies specific product categories despite the noisy nature of review data. We see clear clusters for lighting (lampe, leds), screen protectors (folie, anbringen), and phone cases (hülle, rückseite).

However, we see the limitations of working with raw review data. Unlike the clean news dataset, these topics are heavily polluted by **sentiment words** (`schlecht`, `zurück`) and morphological variations (e.g., `folie` vs. `folien`). While KeyBERT captures the semantic core of the topic, the lack of lemmatization means the model treats singular and plural forms as completely different keywords, wasting valuable space in the topic representation.


#### **Visualization of German topic space**

We visualize the German topics in a 2D projection to see how well-separated they are. In the bar chart, we can inspect particular words in those clusters along with their significance.

In [72]:
fig_map = topic_model_german.visualize_topics()
fig_map.show()

fig_bar = topic_model_german.visualize_barchart(top_n_topics=10)
fig_bar.show()

#### **Inspecting computed French topics**

We repeat the same steps for the inspection of the topics generated by the French model by building a clean topic overview table with interpretable keyword representations.

In [73]:
info_df = topic_model_french.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(6))

,Topic,Count,Top Keywords
1,0,20410,"produit, reçu, recevoir, déçu, pourt, voir"
2,1,667,"colis, coli, reçu, recu, difficil, recevoir"
3,2,170,"recu, reçu, jam, reçut, recommand, reçus"
4,3,60,"fonction, fonctionnent, fonctionnr, fonctionnet, bouton, défaut"
5,4,27,"insect, pest, extérieur, lamp, indésir, moustiquair"
6,5,26,"correspond, descript, attent, conform, compatibl, object"



The French model demonstrates a strong ability to identify specific hardware niches, even within a noisy dataset. We see distinct, high-quality clusters for power accessories (Topic 1: `chargeur`, `batterie`), tablets (Topic 3: `ipad`, `tablette`), and even electronic cigarettes (Topic 5: `cigare`, `électronique`).

However, the noise in the dataset is maybe even more aggressive here than in the German dataset. Topic 0 and Topic 2 are junk clusters dominated by high-frequency stop words like `pas` (not), `fait` (done/makes), and `autre` (other). Unlike standard nouns, these words don't describe a product; they describe the act of reviewing. Furthermore, Topic 4 acts as a sentiment trap, clustering reviews based solely on the word `vraiment` (really/truly).

#### **Visualization of the French topic space**

Here, we visualize the French topic landscape using the same projections and bar chart as for the German topics. This allows us to compare it qualitatively to the German topics.

In [74]:
fig_map = topic_model_french.visualize_topics()
fig_map.show()

fig_bar = topic_model_french.visualize_barchart(top_n_topics=10)
fig_bar.show()

In [64]:
from calculate_t_coherence_and_diversity import evaluate_bertopic_pmi

_, npm_german, div_german = evaluate_bertopic_pmi(
    topic_model=topic_model_german,
    docs=stemmed_german,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="German"
)

_, npm_french, div_french = evaluate_bertopic_pmi(
    topic_model=topic_model_french,
    docs=stemmed_french,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="French"
)

results_df = pd.DataFrame({
    "Metric": ["Coherence (NPMI)", "Diversity"],
    "German": [npm_german, div_german],
    "French": [npm_french, div_french],
})
display(results_df)

German Model - NPMI: 0.2347
German Model - Diversity: 0.3600
French Model - NPMI: 0.1016
French Model - Diversity: 0.3067


,Metric,German,French
0,Coherence (NPMI),0.234705,0.101610
1,Diversity,0.360000,0.306667


# Evaluation

This project aimed to replicate the success of our English News experiment on a more challenging, multilingual dataset of Amazon Reviews. While we successfully extracted interpretable product categories for both German and French, the quantitative metrics reveal that it was far less successful. The coherence scores for both languages are between **0.10 and 0.14**, which is significantly lower than the ~0.40 we achieved with the BBC News dataset.

These lower scores do not necessarily mean the model failed, they rather highlight the nature of the review data. Reviews are short, informal, and heavily saturated with sentiment (e.g., "bad", "perfect") which makes them harder to work with in topics extraction. While the KeyBERT-Inspired representation successfully identified specific items like "LEDs" or "iPads," it could not fully overcome the noise of raw text.

To bridge the gap between the score of **0.14** and our previous benchmark of **0.45**, future iterations would require a strict lemmatization pipeline to merge morphological variants (e.g., `folie` vs. `folien`) and a custom stopword list to filter out the generic transaction language that currently dominates the largest clusters.